In [ ]:
import cv2

# Configurar el clasificador de fondo
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Configurar el detector de contornos
contour_detector = cv2.SimpleBlobDetector_create()

# Abrir el video
cap = cv2.VideoCapture("video_tiburon_2.mp4")

# Definir una región de interés (ROI)
roi = (50, 50, 900, 600)

while True:
    # Leer un fotograma del video
    ret, frame = cap.read()

    if not ret:
        break

    # Aplicar sustracción de fondo solo en la región de interés
    fg_mask = bg_subtractor.apply(frame)
    fg_mask_roi = fg_mask[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]

    # Aplicar umbral para resaltar objetos en movimiento
    _, thresh = cv2.threshold(fg_mask_roi, 50, 255, cv2.THRESH_BINARY)

    # Encontrar contornos en la imagen umbralizada
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos por área para eliminar pequeños artefactos
    min_contour_area = 500
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Dibujar cuadrados alrededor de los tiburones
    for contour in valid_contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Ajustar las coordenadas al espacio original
        x += roi[0]
        y += roi[1]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Tiburon", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar el resultado
    cv2.imshow("Seguimiento de Tiburones", frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()
